# How to get the HiPS from keywords and load it in Aladin 

In [1]:
keywords = ["Mars"]

In [ ]:
#http://alasky.cds.unistra.fr/MocServer/query?expr=hips_frame!=equatorial,galactic,ecliptic&hips_frame=*&dataproduct_subtype=color&obs_regime,client_category=*/Mars&hips_service_url=*&get=record&fmt=json
# Check if obs_description exists and if it contains 'thermal' or other unwanted keywords